
# TensorFlow Object Detection: Обучение модели

https://github.com/tensorflow/models/tree/master/research/object_detection

### Монтирование Google Drive
Для данного демо нам понадобятся несколько ноутбуков, которые должны работать с одними и теме же данными. Поэтому, чтобы все ноутбуки имели доступ к нужным данным, нам будет необходимо подключить диск Google Drive и сохранять все данные на нём (включая данные, скачанные из интернета).

Для монтирования диска нужно выполнить данный блок, перейти по ссылке, получить код, скопировать его в поле ниже (в этом блоке) и нажать Enter

После монтирования диск будет находиться здесь: `/content/drive/My Drive`

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Рабочая директория
Все данные будем хранить в директории `/content/drive/My Drive/tf_od_demo` (TensorFlow Object Detection Demo)

Директория должна быть уже создана (в предыдущем ноутбуке)

In [0]:
%cd "/content/drive/My Drive/tf_od_demo"

### Загрузка предобученной модели
Во время обучения мы будем делать fine-tuning предобученной модели. Для этого необходимо загрузить веса соответствующей модели `ssd_mobilenet_v1_coco`. Из названия следует, что детекционная архитектура -- `SSD`, базовая CNN модель -- `MobileNet_v1`, предобучена на датасете COCO.

Этот шаг нужно сделать один раз (не повторять, если модель уже была скачана ранее).

In [0]:
if True:
    !wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
    !tar -xzf ssd_mobilenet_v1_coco_2018_01_28.tar.gz

### Подготовка данных для обучения
Ранее мы подготовили файл `train_data.record`, в котором содержится обучающий датасет.
Кроме него для обучения необходимо подготовить еще два файла (они также находятся в папке `my_data`)

* `label_map.pbtxt` (в нашем случае `cube_label_map.pbtxt`) - файл с описанием классов. В нашем случае у нас всего один класс.

* `pipeline.config` - файл, содержищий различне гиперпарамтеры и настройки обучения. Важные параметры: `num_classes`, `fine_tune_checkpoint`, `label_map_path`, `input_path`. В данном примере и для обучения и для теста используется один и тот же датасет (см. блоки параметров `train_input_reader`, `eval_input_reader`)

### Запуск обучения
После того, как мы подготовили все данные, запустить обучение очень просто: надо просто запустить скрипт `model_main.py` и передать ему соответствующие параметры.

*   Путь к файлу `pipeline.config`
*   Директорию, для записи выхода обучения (`model_dir`)
*   Количество итераций обучения (`num_train_steps`)



In [0]:
%%time

if False:
    # Delete output directory
    !rm -rf my_data/output

!export PYTHONPATH=$PYTHONPATH:models/research:models/research/slim ; python models/research/object_detection/model_main.py \
    --pipeline_config_path=my_data/pipeline.config \
    --model_dir=my_data/output \
    --num_train_steps=10000 \
    --alsologtostderr

### Заморозка модели
Для того, чтобы использовать обученную модель в режиме инференса, её нужно "заморозить". Другими словами - подготовить к инференсу. Для этого надо запустить скрипт `export_inference_graph.py` и передать соответствующие парамтеры.

* Путь к файлу `pipeline.config`
* Путь к весам обученной модели (`trained_checkpoint_prefix`)
* Путь к директории для сохранения замороженной модели (`output_directory`)

In [0]:
!export PYTHONPATH=$PYTHONPATH:models/research:models/research/slim ; python models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=my_data/pipeline.config \
    --trained_checkpoint_prefix=my_data/output/model.ckpt-10000 \
    --output_directory=my_data/output/frozen/